<a href="https://colab.research.google.com/github/jkauffm4/Intro-to-ML/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
device = torch.device('cuda')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True)
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True)

100%|██████████| 170M/170M [00:02<00:00, 79.4MB/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
class Q1A_Net(torch.nn.Module):
    def __init__(self):
        super(Q1A_Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(2)
        self.fc1 = torch.nn.Linear(8 * 8 * 8, 32)
        self.act4 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
Q1A_model = Q1A_Net()

In [ ]:
from torchvision import transforms
cifar10 = datasets.CIFAR10(data_path, train = True, download = True)
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True)
img, label = cifar10[99]
to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
imgs = torch.stack([to_tensor(img) for img, label in cifar10], dim=3)
imgs.view(3, -1).mean(dim=1)
imgs.view(3, -1).std(dim=1)

Files already downloaded and verified
Files already downloaded and verified


tensor([0.2470, 0.2435, 0.2616])

In [ ]:
transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
cifar10 = datasets.CIFAR10(data_path, train = True, download = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train = False, download = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
img, _ = cifar10[99]
imgs = torch.stack([img for img, _ in cifar10], dim=3)
#imgs = imgs.to(device)

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
Q1A_model = Q1A_Net().to(device=device)
learning_rate = 1e-2
optimizer = torch.optim.Adam(Q1A_model.parameters(), lr = learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
n_epochs = 200
for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    imgs = imgs.to(device=device)
    labels = labels.to(device=device)
    outputs = Q1A_model(imgs)
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.581403
Epoch: 1, Loss: 1.644853
Epoch: 2, Loss: 1.216052
Epoch: 3, Loss: 1.664903
Epoch: 4, Loss: 1.474846
Epoch: 5, Loss: 1.180468
Epoch: 6, Loss: 1.479889
Epoch: 7, Loss: 0.966518
Epoch: 8, Loss: 2.150442
Epoch: 9, Loss: 1.074386
Epoch: 10, Loss: 0.955610
Epoch: 11, Loss: 1.238214
Epoch: 12, Loss: 1.580573
Epoch: 13, Loss: 1.064871
Epoch: 14, Loss: 1.142614
Epoch: 15, Loss: 1.455808
Epoch: 16, Loss: 0.565571
Epoch: 17, Loss: 1.624984
Epoch: 18, Loss: 1.543944
Epoch: 19, Loss: 0.650686
Epoch: 20, Loss: 1.322194
Epoch: 21, Loss: 1.240772
Epoch: 22, Loss: 1.759160
Epoch: 23, Loss: 0.643133
Epoch: 24, Loss: 0.834033
Epoch: 25, Loss: 1.055036
Epoch: 26, Loss: 0.962420
Epoch: 27, Loss: 1.308399
Epoch: 28, Loss: 1.024362
Epoch: 29, Loss: 1.315783
Epoch: 30, Loss: 0.728063
Epoch: 31, Loss: 1.304243
Epoch: 32, Loss: 2.114763
Epoch: 33, Loss: 1.025118
Epoch: 34, Loss: 1.074024
Epoch: 35, Loss: 0.915140
Epoch: 36, Loss: 1.055834
Epoch: 37, Loss: 1.412099
Epoch: 38, Loss: 0.571

KeyboardInterrupt: 

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = cifar10_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.094920


In [ ]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = cifar10_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.093500


In [ ]:
class Q1B_Net(torch.nn.Module):
    def __init__(self):
        super(Q1B_Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(2)
        self.conv3 = torch.nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.act3 = torch.nn.ReLU()
        self.pool3 = torch.nn.MaxPool2d(2)
        self.fc1 = torch.nn.Linear(4 * 4 * 4, 32)
        self.act4 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 4 * 4 * 4)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
cifar10_model = Q1B_Net()
learning_rate = 1e-2
optimizer = torch.optim.Adam(cifar10_model.parameters(), lr = learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
n_epochs = 200
for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    outputs = cifar10_model(imgs)
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward
    optimizer.step()
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.307894
Epoch: 1, Loss: 2.300971
Epoch: 2, Loss: 2.298497
Epoch: 3, Loss: 2.310263
Epoch: 4, Loss: 2.298687
Epoch: 5, Loss: 2.270029
Epoch: 6, Loss: 2.308037
Epoch: 7, Loss: 2.299601
Epoch: 8, Loss: 2.329780
Epoch: 9, Loss: 2.314899
Epoch: 10, Loss: 2.305883
Epoch: 11, Loss: 2.295322
Epoch: 12, Loss: 2.360054
Epoch: 13, Loss: 2.353308
Epoch: 14, Loss: 2.270508
Epoch: 15, Loss: 2.335371
Epoch: 16, Loss: 2.259885
Epoch: 17, Loss: 2.288702
Epoch: 18, Loss: 2.315046
Epoch: 19, Loss: 2.290932
Epoch: 20, Loss: 2.355801
Epoch: 21, Loss: 2.309774
Epoch: 22, Loss: 2.277247
Epoch: 23, Loss: 2.280517
Epoch: 24, Loss: 2.321770
Epoch: 25, Loss: 2.341521
Epoch: 26, Loss: 2.345055
Epoch: 27, Loss: 2.328620
Epoch: 28, Loss: 2.298835
Epoch: 29, Loss: 2.286715
Epoch: 30, Loss: 2.250443
Epoch: 31, Loss: 2.260070
Epoch: 32, Loss: 2.298206
Epoch: 33, Loss: 2.312478
Epoch: 34, Loss: 2.346912
Epoch: 35, Loss: 2.324882
Epoch: 36, Loss: 2.339622
Epoch: 37, Loss: 2.280136
Epoch: 38, Loss: 2.248

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = cifar10_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.099820


In [ ]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = cifar10_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.100100


In [ ]:
class ResNetBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.act1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)

        self.shortcut = torch.nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
                torch.nn.BatchNorm2d(out_channels)
            )
    def forward(self, x):
        out = self.act1(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.nn.ReLU()(out)
        return out

In [ ]:
class Q2A_Net(torch.nn.Module):
    def __init__(self):
        super(Q2A_Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.act1 = torch.nn.ReLU()

        self.resnet1 = ResNetBlock(16, 16)
        self.resnet2 = ResNetBlock(16, 16)
        self.resnet3 = ResNetBlock(16, 32)
        self.resnet4 = ResNetBlock(32, 32)
        self.resnet5 = ResNetBlock(32, 64)
        self.resnet6 = ResNetBlock(64, 64)
        self.resnet7 = ResNetBlock(64, 128)
        self.resnet8 = ResNetBlock(128, 128)
        self.resnet9 = ResNetBlock(128, 256)
        self.resnet10 = ResNetBlock(256, 256)

        self.pool = torch.nn.MaxPool2d(2)
        self.fc1 = torch.nn.Linear(256 * 2 * 2, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        out = self.act1(self.bn1(self.conv1(x)))
        out = self.pool(out)

        out = self.resnet1(out)
        out = self.pool(out)
        out = self.resnet2(out)
        out = self.pool(out)
        out = self.resnet3(out)
        out = self.pool(out)
        out = self.resnet4(out)
      #  out = self.pool(out)
        out = self.resnet5(out)
     #   out = self.pool(out)
        out = self.resnet6(out)
      #  out = self.pool(out)
        out = self.resnet7(out)
       # out = self.pool(out)
        out = self.resnet8(out)
      #  out = self.pool(out)
        out = self.resnet9(out)
      #  out = self.pool(out)
        out = self.resnet10(out)

        out = out.view(-1, 256 * 2 * 2)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
Q2A_model = Q2A_Net().to(device=device)
learning_rate = 1e-2
optimizer = torch.optim.Adam(Q2A_model.parameters(), lr = learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
n_epochs = 200
for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    imgs = imgs.to(device=device)
    labels = labels.to(device=device)
    outputs = Q2A_model(imgs)
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.630592
Epoch: 1, Loss: 0.916337
Epoch: 2, Loss: 2.246473
Epoch: 3, Loss: 0.932922
Epoch: 4, Loss: 0.989297
Epoch: 5, Loss: 0.597788
Epoch: 6, Loss: 1.016242
Epoch: 7, Loss: 0.729669
Epoch: 8, Loss: 0.157600
Epoch: 9, Loss: 1.026040
Epoch: 10, Loss: 0.771464
Epoch: 11, Loss: 0.346271
Epoch: 12, Loss: 0.615090


In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = Q2A_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = Q2A_model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))